In [1]:
# Setup
# `jupyter nbextension enable --py --sys-prefix ipyvuetify`#
# start: `voila .\smart_grids.ipynb`
# icons here https://vuetifyjs.com/en/features/icon-fonts/#material-design-icons


In [2]:
import plotly.graph_objects as go
import ipyvuetify as v


In [3]:
def update_stepper(widget, *args):
    stepper.v_model+=widget.direction
    try:
        widget.step.complete=True
    except: 
        pass

In [4]:
class BackBtn(v.Btn):
    def __init__(self):
        self.direction=-1
        super().__init__(children="Back")
        self.on_event("click",update_stepper)

In [5]:
class NextBtn(v.Btn):
    def __init__(self,container,step,disabled=True):
        self.step=step
        self.direction=1
        self.container=container
        container.next_btn=self
        super().__init__(children="Next")
        self.on_event("click",update_stepper)
        self.disabled=disabled

In [6]:
import address_widget
address_container=address_widget.Address()
# address_container
address_step_number = 1
address_step = v.StepperStep(step=address_step_number, children=["Address"], complete=False)
address_step_content = v.StepperContent(step=address_step_number,children=[address_container, NextBtn(address_container,address_step)])

In [7]:
import battery_widget
battery_container = battery_widget.Battery()
# battery_container
battery_step_number = 2
battery_step = v.StepperStep(step=battery_step_number, children=["Battery"], complete=False)
battery_step_content = v.StepperContent(step=battery_step_number,children=[battery_container, BackBtn(), NextBtn(battery_container,battery_step)])

In [8]:
from util.get_photovoltaic_data import get_photovoltaic_data


def get_pv_power():
    global pv_data
    pv_data= [
        {"time": data["time"], "PV system power": data["P"]}
        for data in get_photovoltaic_data(
            latitude=address_container.latitude.v_model,
            longitude=address_container.longitude.v_model,
            pv_tech=pv_container.pv_technology,
            pv_peak_power=pv_container.pv_peak_power.v_model,
            pv_loss=pv_container.pv_loss.v_model,
            pv_angle=pv_container.pv_slope.v_model,
            pv_azimuth=pv_container.pv_azimuth.v_model,
        )["outputs"]["hourly"]
    ]

In [31]:
# TODO do proper
pv_data=[{"time":'20150101:0010',"PV system power":2}]
def get_data_fig(data):
    return go.Figure(
        data=[go.Bar(x=[o["time"] for o in data], y=[o["PV system power"] for o in data])],
        layout_title_text="A Figure Displaying Itself",
    )


In [32]:
import pv_widget
pv_container = pv_widget.Photovoltaic()
pv_step_number = 3
pv_step = v.StepperStep(step=pv_step_number, children=["Photvoltaic"], complete=False)
pv_next_button=NextBtn(pv_container,pv_step)
def update_stepper_and_calculate(widget, *args):
    stepper.v_model+=widget.direction
    try:
        widget.step.complete=True
    except: 
        pass
    get_data_fig()
pv_next_button.on_event("click", update_stepper_and_calculate)
pv_next_button.children=["calculate"]
pv_step_content = v.StepperContent(step=pv_step_number,children=[pv_container, BackBtn(), pv_next_button])

In [33]:
pv_power_container=v.Container(children=["graph here"])
pv_power=NextBtn(pv_container,pv_step)
pv_power_step_number=4
pv_power_step = v.StepperStep(step=pv_power_step_number, children=["Photvoltaic Power"], complete=True)
pv_power_step_content = v.StepperContent(step=pv_power_step_number,children=[pv_power_container, BackBtn(), NextBtn(pv_power_container,pv_power_step,disabled=False)])

In [34]:
import consumers_widget
consumer_container=consumers_widget.ConsumerTable()

consumer_step_number = 5
consumer_step = v.StepperStep(step=consumer_step_number, children=["Consumers"], complete=False)
consumer_step_content = v.StepperContent(step=consumer_step_number,children=[consumer_container, BackBtn(), NextBtn(consumer_container,consumer_step)])

In [40]:

stepper = v.Stepper(
    v_model=1,
    children=[
        v.StepperHeader(
            children=[
                address_step,
                v.Divider(),
                battery_step,
                v.Divider(),
                pv_step,
                v.Divider(),
                pv_power_step,
                v.Divider(),
                consumer_step
            ]
        ),
        v.StepperItems(
            children=[
                address_step_content,
                battery_step_content,
                pv_step_content,
                pv_power_step_content,
                consumer_step_content
            ]
        ),
    ],
)
stepper


Stepper(children=[StepperHeader(children=[StepperStep(children=['Address'], complete=True, step=1.0), Divider(…

In [ ]:
v.Card(children=["hshj"])

Card(children=['hshj'])